In [ ]:
pip install --upgrade --force-reinstall scipy==1.10.1


In [1]:
!pip install ultralytics --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00a 0:00:01


In [ ]:
import os
import cv2
import torch
import random
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

print("✅ Albumentations", A.__version__)
print("✅ Torch", torch.__version__)

# =========================================================
# ✅ GLOBAL SEED FOR FULL REPRODUCIBILITY
# =========================================================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# =========================================================
# ✅ CCTV Augmentation Pipeline (with bounding-box safety)
# =========================================================
train_transform = A.Compose([
    A.RandomResizedCrop(size=(640, 640), scale=(0.7, 1.0), ratio=(0.75, 1.33), p=1.0),

    # Geometric
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.ShiftScaleRotate(
        shift_limit=0.05,
        scale_limit=0.1,
        rotate_limit=15,
        border_mode=cv2.BORDER_CONSTANT,
        p=0.8
    ),

    # Lighting & color (CCTV realism)
    A.RandomBrightnessContrast(p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=15, p=0.6),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.4),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.3),

    # Blur and noise (CCTV quality loss)
    A.MotionBlur(p=0.3),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.GaussNoise(var_limit=(5, 25), p=0.3),

    # Occlusion / fog simulation
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.4),

    ToTensorV2()
],
    bbox_params=A.BboxParams(
        format='yolo',                 # YOLO (x_center, y_center, w, h)
        label_fields=['class_labels'],  # Keep labels with boxes
        min_visibility=0.15            # Removes tiny leftover objects after augmentation
    )
)


# =========================================================
# ✅ Load YOLOv9m Model
# =========================================================
model = YOLO("yolov9m.pt")

# Light built-in augmentations (as you set before)
model.overrides['augment'] = True
model.overrides['rect'] = False
model.overrides['mosaic'] = 0.3
model.overrides['mixup'] = 0.2
model.overrides['copy_paste'] = 0.0


# =========================================================
# ✅ Dataset + Save Directory
# =========================================================
data_yaml = "/kaggle/input/vioelnceweapondetectiond/data.yaml"
save_dir = "/kaggle/working/yolov9m_cctv_aug_runs"


# =========================================================
# ✅ Hook Albumentations into YOLO Dataloader
# =========================================================
def custom_dataloader_hook(dataset):
    dataset.transforms = train_transform
    return dataset

model.add_callback("on_fit_epoch_start",
                   lambda trainer: custom_dataloader_hook(trainer.train_loader.dataset))


# =========================================================
# ✅ Train Model
# =========================================================
results = model.train(
    data=data_yaml,
    epochs=200,
    imgsz=640,
    batch=16,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=save_dir,
    name="yolov9m_cctv_aug",
    workers=4,
    lr0=0.001,
    optimizer="SGD",
    pretrained=True,
    verbose=True,
    val=True,
    exist_ok=True,
    patience=15
)


# =========================================================
# ✅ Evaluate Model
# =========================================================
metrics = model.val()
print("\n📊 Validation Metrics:")
try:
    print(f"Precision:   {metrics.box.pr:.4f}")
    print(f"Recall:      {metrics.box.re:.4f}")
    print(f"mAP50:       {metrics.box.map50:.4f}")
    print(f"mAP50-95:    {metrics.box.map:.4f}")
except:
    print(metrics)

print("\n✅ Training completed successfully with YOLOv9m + CCTV augmentations + bbox safety + reproducibility!")


✅ Albumentations 2.0.8
✅ Torch 2.6.0+cu124


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_87/3648390766.py:52: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 25), p=0.3),
/tmp/ipykernel_87/3648390766.py:55: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.4),


Ultralytics 8.3.221 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/vioelnceweapondetectiond/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov9m.pt, momentum=0.937, mosaic=0.3, multi_scale=False, name=yolov9m_cctv_aug, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=15, perspectiv